In [1]:
from torchdrug import  layers, datasets,transforms,core
from torchdrug.core import Registry as R
from torchdrug.layers import geometry

import torch
from torchdrug import data

from torch_scatter import scatter_add
import torch.nn as nn
from torchdrug import utils
from torch.utils import checkpoint
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据集获取

In [2]:
EnzymeCommission = R.search("datasets.EnzymeCommission")
PV = R.search("transforms.ProteinView")
trans = PV(view = "residue")
dataset = EnzymeCommission("~/scratch/protein-datasets/", test_cutoff=0.95, 
                           atom_feature="full", bond_feature="full", verbose=1, transform = trans)

# 只保留alpha碳的简化格式
graph_construction_model = layers.GraphConstruction(node_layers=[geometry.AlphaCarbonNode()], 
                                                    edge_layers=[geometry.SpatialEdge(radius=10.0, min_distance=5),
                                                                 geometry.KNNEdge(k=10, min_distance=5),
                                                                 geometry.SequentialEdge(max_distance=2)
                                                                 ],
                                                    edge_feature="gearnet"
                                                    )


graphs = dataset[:2]
graphs = [element["graph"] for element in graphs]
graphs = data.Protein.pack(graphs)
print("\n\n")
graph = graph_construction_model(graphs)
print(graph)

18:24:09   Extracting /home/xiaotong/scratch/protein-datasets/EnzymeCommission.zip to /home/xiaotong/scratch/protein-datasets


Loading /home/xiaotong/scratch/protein-datasets/EnzymeCommission/enzyme_commission.pkl.gz: 100%|██████████| 18716/18716 [00:46<00:00, 406.51it/s]





PackedProtein(batch_size=2, num_atoms=[185, 415], num_bonds=[3754, 8999], num_residues=[185, 415])


In [3]:
#graph = graph.to(device)

print(graph.num_nodes)
print(graph.batch_size)
print(graph.edge_list)

tensor([185, 415])
2
tensor([[ 95,  96,   5],
        [109, 110,   5],
        [108, 109,   5],
        ...,
        [438, 470,   0],
        [489, 470,   0],
        [493, 470,   0]])


# 关系卷积神经网络，获取多个不同的嵌入

In [4]:
class relationalGraph(layers.MessagePassingBase):
    
    def __init__(self, input_dim, output_dim, num_relation, edge_input_dim=None, batch_norm=False, activation="relu"):
        super(relationalGraph, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_relation = num_relation
        self.edge_input_dim = edge_input_dim

        if batch_norm:
            self.batch_norm = nn.BatchNorm1d(output_dim)
        else:
            self.batch_norm = None
        if isinstance(activation, str):
            self.activation = getattr(F, activation)
        else:
            self.activation = activation

        self.self_loop = nn.Linear(input_dim, output_dim)
        self.linear = nn.Linear(input_dim, output_dim)
        if edge_input_dim:
            self.edge_linear = nn.Linear(edge_input_dim, input_dim)
        else:
            self.edge_linear = None
            
    def trans(self, A, graph):
    
        Degree_inv_sqrt = torch.diag(torch.pow(torch.sum(A, dim=1), -0.5))
        A_norm = torch.mm(torch.mm(Degree_inv_sqrt, A), Degree_inv_sqrt)
        
        n_rel = graph.num_relation
        n = A_norm.size(0)
        n_rel = n_rel.item()  # 将 n_rel 从 Tensor 转换为 int
        assert n % n_rel == 0, "n must be divisible by n_rel"
        
        block_size = n // n_rel
        
        # 初始化一个张量来存储累加结果
        accumulated = torch.zeros_like(A_norm[:block_size])
        
        # 将后面的所有块累加到第一块
        for i in range(n_rel):
            accumulated += A_norm[i * block_size: (i + 1) * block_size]
        
        # 用累加后的第一块替换原始矩阵的第一块
        A_trans = accumulated
    
        return A_trans

    def message_and_aggregate(self, graph, input, new_edge_list):
        assert graph.num_relation == self.num_relation
        device = input.device  # Ensure device consistency
        
        if new_edge_list is None:
            node_in, node_out, relation = graph.edge_list.t().to(device)
            node_out = node_out * self.num_relation + relation
        
            edge_weight = torch.ones_like(node_out)
            degree_out = scatter_add(edge_weight, node_out, dim_size=graph.num_node * graph.num_relation)
            degree_out = degree_out
            edge_weight = edge_weight / degree_out[node_out]
            adjacency = utils.sparse_coo_tensor(torch.stack([node_in, node_out]), edge_weight,
                                                (graph.num_node, graph.num_node * graph.num_relation))
            update = torch.sparse.mm(adjacency.t(), input)
        
        else:
            adjacency = self.trans(new_edge_list, graph).to(device)
            update = torch.mm(adjacency.t().to(device), input.to(device))
        
        if self.edge_linear:
            edge_input = graph.edge_feature.float().to(device)
            edge_input = self.edge_linear(edge_input)
            edge_weight = edge_weight.unsqueeze(-1).to(device)
            edge_update = scatter_add(edge_input * edge_weight, node_out, dim=0,
                                      dim_size=graph.num_node * graph.num_relation)
            update += edge_update

        return update

    def combine(self, input, update):
        # 自环特征
        device = input.device
        self.linear.to(device)  # Ensure the linear layers are on the correct device
        self.self_loop.to(device)
        input = input.repeat(self.num_relation, 1).to(device)
        loop_update = self.self_loop(input).to(device)
        
        output = self.linear(update)+loop_update
        if self.batch_norm:
            output = self.batch_norm(output)
        if self.activation:
            output = self.activation(output)
        return output
    
    def forward(self, graph, input, new_edge_list=None):
        
        if self.gradient_checkpoint:
            update = checkpoint.checkpoint(self._message_and_aggregate, *graph.to_tensors(), input)
        else:
            update = self.message_and_aggregate(graph, input, new_edge_list)
        output = self.combine(input, update)
        return output

#### 测试

In [5]:
input_dim = graph.node_feature.shape[-1]
output_dim = 512
num_relations = graph.num_relation

relational_output = relationalGraph(input_dim, output_dim, num_relations)(graph, graph.node_feature.float(), new_edge_list = None)
print(relational_output)
print("output: ", relational_output.shape)

tensor([[0.0387, 0.2849, 0.0469,  ..., 0.2518, 0.0481, 0.0000],
        [0.2962, 0.1971, 0.1415,  ..., 0.1966, 0.0247, 0.0000],
        [0.5053, 0.3646, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.4512, 0.3146, 0.0038,  ..., 0.0000, 0.1177, 0.0000],
        [0.5053, 0.3646, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.2075, 0.3166, 0.0000,  ..., 0.0000, 0.0713, 0.0000]],
       grad_fn=<ReluBackward0>)
output:  torch.Size([4200, 512])


# 重连接模块

### 只从点所在的图进行reconnection

In [6]:
import torch

def get_start_end(current, graph):
    """
    根据一维张量 a 生成新的张量 b。
    
    :param a: 输入的一维张量
    :return: 输出的一维张量 b
    """
    # 初始化 b，第一个元素是 0
    segment = graph.num_nodes.repeat(graph.num_relation)
    index = torch.zeros(segment.size(0) + 1, dtype=segment.dtype)
    
    # 计算 b 的每个元素
    for i in range(1, len(index)):
        index[i] = index[i - 1] + segment[i - 1]
    
    # 遍历张量以找到索引值的位置
    for i in range(len(index) - 1):
        if index[i] <= current < index[i + 1]:
            return (index[i].item(), index[i + 1].item())
        elif index[i] == current:
            return (index[i].item(), index[i + 1].item())
    
    # 如果索引值恰好等于张量的最后一个元素
    if current == index[-1]:
        return (index[-1].item(), index[-1].item())
    
    

# 示例使用
start, end = get_start_end(184, graph)
print(start, end)


0 185


In [7]:
def get_start_end2( current, graph):
    segment = graph.num_nodes.repeat(graph.num_relation)
    index = torch.cumsum(segment, dim=0)
    
    # Use torch.searchsorted to find the appropriate segment
    pos = torch.searchsorted(index, current, right=True)

    if pos == 0:
        return (0, index[0].item())
    elif pos >= len(index):
        return (index[-1].item(), index[-1].item())
    else:
        return (index[pos-1].item(), index[pos].item())
    
start, end = get_start_end2(600, graph)
print(start, end)


600 785


### Gumble-softmax采样

In [95]:
import torch
import torch.nn.functional as F

def gumbel_softmax_top_k(logits, tau=0.5, k=1, hard=False):
    """
    Gumbel-Softmax采样方法，每一步都是可微分的，并选择最大的k个元素。
    
    参数:
        logits (torch.Tensor): 输入logits张量，维度为 (batch_size, num_classes)。
        tau (float): Gumbel-Softmax的温度参数，控制平滑程度。
        k (int): 选择最大的k个元素。
        hard (bool): 是否返回硬分类结果。
    
    返回:
        torch.Tensor: Gumbel-Softmax采样结果，维度为 (batch_size, num_classes)。
    """
    # 获取Gumbel分布噪声
    gumbels = -torch.empty_like(logits).exponential_().log()  # 生成Gumbel(0,1)噪声
    gumbels = (logits + gumbels) / tau  # 添加噪声并除以温度参数

    # 计算softmax
    y_soft = F.softmax(gumbels, dim=-1)  # 维度为 (batch_size, num_classes)

    if hard:
        # 硬分类结果：选取原始logits最大的k个位置
        topk_indices = logits.topk(k, dim=-1)[1]  # 获取前k个元素的索引
        y_hard = torch.zeros_like(logits).scatter_(-1, topk_indices, 1.0)  # 生成one-hot向量
        # 使用直通估计器
        y = (y_hard - y_soft).detach() + y_soft
    else:
        y = y_soft

    return y

# 示例用法
if __name__ == "__main__":
    logits = torch.randn(5, 10)  # 维度为 (batch_size=5, num_classes=10)
    tau = 0.5
    k = 3
    hard = True
    samples = gumbel_softmax_top_k(logits, tau, k, hard)
    print(samples)
    samples = gumbel_softmax_top_k(logits, tau, k)
    print(samples)


tensor([[0., 0., 0., 1., 1., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 1., 1.],
        [1., 0., 0., 1., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 1., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 1., 1.]])
tensor([[4.8048e-01, 3.7910e-04, 2.6873e-04, 4.0213e-03, 1.1619e-01, 5.7953e-04,
         3.6010e-01, 6.4904e-04, 1.2300e-02, 2.5035e-02],
        [5.5033e-04, 6.5851e-05, 1.9387e-05, 5.5963e-05, 9.5151e-02, 1.5899e-01,
         5.4683e-05, 1.9378e-01, 5.4991e-01, 1.4249e-03],
        [9.8238e-01, 2.3563e-05, 5.6371e-04, 5.5736e-03, 8.7534e-05, 5.6787e-05,
         8.2759e-03, 2.6395e-03, 7.3919e-05, 3.2825e-04],
        [6.5204e-05, 4.4989e-04, 9.1505e-03, 3.3549e-02, 8.4767e-03, 9.2767e-05,
         4.9709e-03, 1.1033e-03, 9.4183e-01, 3.1603e-04],
        [2.2411e-04, 3.5781e-06, 1.3032e-03, 9.5245e-07, 5.9685e-05, 1.1207e-04,
         5.7596e-05, 5.7783e-03, 9.1864e-01, 7.3822e-02]])


### window self attention + gumble softmax

In [8]:
import torch
from torch import nn
import torch.nn.functional as F

class Rewirescorelayer(nn.Module):
    def __init__(self, in_features, out_features, num_heads, window_size, k, temperature=0.5, dropout=0.1):
        super(Rewirescorelayer, self).__init__()
        self.num_heads = num_heads
        self.out_features = out_features
        self.window_size = window_size
        self.temperature = temperature
        self.dropout = nn.Dropout(dropout)
        self.k = k

        self.query = nn.Linear(in_features, out_features * num_heads)
        self.key = nn.Linear(in_features, out_features * num_heads)
        self.value = nn.Linear(in_features, out_features * num_heads)
        self.scale = torch.sqrt(torch.FloatTensor([out_features])).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

    def get_start_end(self, current, graph):
        segment = graph.num_nodes.repeat(graph.num_relation)
        index = torch.cumsum(segment, dim=0)
        
        # Use torch.searchsorted to find the appropriate segment
        pos = torch.searchsorted(index, current, right=True)

        if pos == 0:
            return (0, index[0].item())
        elif pos >= len(index):
            return (index[-1].item(), index[-1].item())
        else:
            return (index[pos-1].item(), index[pos].item())

    def gumbel_softmax_top_k(self, logits, tau=1.0, hard=False):
        gumbels = -torch.empty_like(logits).exponential_().log()
        gumbels = (logits + gumbels) / tau

        y_soft = F.softmax(gumbels, dim=-1)

        if hard:
            topk_indices = logits.topk(self.k, dim=-1)[1]
            y_hard = torch.zeros_like(logits).scatter_(-1, topk_indices, 1.0)
            y = (y_hard - y_soft).detach() + y_soft
        else:
            y = y_soft

        return y

    def forward(self, graph, node_features):
        device = node_features.device
        num_nodes = node_features.size(0)
        half_window = self.window_size // 2

        Q = self.query(node_features).view(num_nodes, self.num_heads, self.out_features)
        K = self.key(node_features).view(num_nodes, self.num_heads, self.out_features)

        output = torch.zeros(num_nodes, num_nodes, device=device)

        start_end_indices = [self.get_start_end(i, graph) for i in range(num_nodes)]

        # Precompute K_windows
        K_windows_padded = torch.zeros((num_nodes, self.window_size, self.num_heads, self.out_features), device=device)
        for i in range(num_nodes):
            start_index, end_index = start_end_indices[i]
            start = max(start_index, i - half_window)
            end = min(end_index, i + half_window)
            K_window = K[start:end]
            K_windows_padded[i, :K_window.size(0)] = K_window

        #scores = torch.einsum("nhd,nmhd->nhm", Q, K_windows_padded) / self.scale        # (num_nodes, num_heads, window_size), nhm
        batch_size = 100  # 根据GPU内存调整批量大小
        scores_list = []

        for i in range(0, num_nodes, batch_size):
            end_i = min(i + batch_size, num_nodes)
            scores_batch = torch.einsum("nhd,nmhd->nhm", Q[i:end_i], K_windows_padded[i:end_i]) / self.scale
            scores_list.append(scores_batch)

        # 将所有批次的得分合并为一个张量
        scores = torch.cat(scores_list, dim=0)
        
        scores = scores / self.temperature   # nhm

        attention_weights = F.softmax(scores, dim=-1).mean(dim=1)   # (num_nodes, window_size)
        

        for i in range(num_nodes):
            start_index, end_index = start_end_indices[i]
            start = max(start_index, i - half_window)
            end = min(end_index, i + half_window )
            output[i, start:end] = attention_weights[i, :end-start]

        edge_list = self.gumbel_softmax_top_k(output, self.temperature, self.k)

        return edge_list

#### 测试

In [13]:

input_dim = relational_output.shape[-1]
output_dim = 1024
num_heads = 8
window_size = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
k = 5


relational_output = relational_output.to(device)
module = Rewirescorelayer(input_dim, output_dim, num_heads, window_size, k).to(device)
attn_output = module(graph, relational_output)

In [14]:
print(attn_output.shape)
print(attn_output)
#a = attn_output[1200:, 599:1199]
indices = torch.nonzero(attn_output, as_tuple=True)
print(indices)

torch.Size([4200, 4200])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AddBackward0>)
(tensor([   0,    0,    0,  ..., 4199, 4199, 4199], device='cuda:0'), tensor([   7,   10,   12,  ..., 4193, 4194, 4195], device='cuda:0'))


# Diffusion模块

### 计算degree矩阵，变换adjacent matrix形式

In [9]:
def trans(A, graph):
    
    Degree_inv_sqrt = torch.diag(torch.pow(torch.sum(A, dim=1), -0.5))
    A_norm = torch.mm(torch.mm(Degree_inv_sqrt, A), Degree_inv_sqrt)
    
    n_rel = graph.num_relation
    n = A_norm.size(0)
    n_rel = n_rel.item()  # 将 n_rel 从 Tensor 转换为 int
    assert n % n_rel == 0, "n must be divisible by n_rel"
    
    block_size = n // n_rel
    
    # 初始化一个张量来存储累加结果
    accumulated = torch.zeros_like(A_norm[:block_size])
    
    # 将后面的所有块累加到第一块
    for i in range(n_rel):
        accumulated += A_norm[i * block_size: (i + 1) * block_size]
    
    # 用累加后的第一块替换原始矩阵的第一块
    A_trans = accumulated
    
    
    
    return A_trans

A_norm = trans(attn_output, graph)
print(A_norm)
print(A_norm.shape)

indices = torch.nonzero(A_norm, as_tuple=True)
print(indices)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
update = torch.mm(A_norm.t(), graph.node_feature.to(device).to(torch.float))
print(update.shape)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AddBackward0>)
torch.Size([600, 4200])
(tensor([  0,   0,   0,  ..., 599, 599, 599], device='cuda:0'), tensor([   3,    6,    8,  ..., 4193, 4195, 4196], device='cuda:0'))
torch.Size([4200, 21])


### Rewired_gearnet 用于diffusion模块

In [15]:
class RewireGearnet(nn.Module):
    gradient_checkpoint = False

    def __init__(self, input_dim, output_dim, num_relation, edge_input_dim=None, batch_norm=False, activation="relu"):
        super(RewireGearnet, self).__init__()
        self.num_relation = num_relation
        self.input_dim = input_dim
        self.output_dim = output_dim

        self.linear = nn.Linear(num_relation * input_dim, output_dim)
        self.self_loop = nn.Linear(input_dim, output_dim)
        self.batch_norm = nn.BatchNorm1d(output_dim) if batch_norm else None
        self.activation = getattr(F, activation) if activation else None
        self.edge_linear = nn.Linear(edge_input_dim, output_dim) if edge_input_dim else None

    def trans(self, A, graph):
        n_rel = graph.num_relation
        n = A.size(0)
        n_rel = n_rel.item()  # 将 n_rel 从 Tensor 转换为 int
        assert n % n_rel == 0, "n must be divisible by n_rel"
        
        block_size = n // n_rel
        
        # 初始化一个张量来存储累加结果
        accumulated = torch.zeros_like(A[:block_size])
        
        # 将后面的所有块累加到第一块
        for i in range(n_rel):
            accumulated += A[i * block_size: (i + 1) * block_size]
        
        # 用累加后的第一块替换原始矩阵的第一块
        A_trans = accumulated
    
        return A_trans

    def message_and_aggregate(self, graph, input, new_edge_list=None):
        assert graph.num_relation == self.num_relation

        device = input.device  # Ensure device consistency

        if new_edge_list is None:
            node_in, node_out, relation = graph.edge_list.t().to(device)
            node_out = node_out * self.num_relation + relation
            adjacency = torch.sparse_coo_tensor(
                torch.stack([node_in, node_out]),
                graph.edge_weight.to(device),
                (graph.num_node, graph.num_node * graph.num_relation),
                device=device
            )
            update = torch.sparse.mm(adjacency.t(), input)
        else:
            adjacency = self.trans(new_edge_list, graph).to(device)
            update = torch.mm(adjacency.t(), input.to(device))
        
        if self.edge_linear:
            edge_input = graph.edge_feature.float().to(device)
            edge_input = self.edge_linear(edge_input)
            edge_weight = graph.edge_weight.unsqueeze(-1).to(device)
            edge_update = scatter_add(
                edge_input * edge_weight, node_out, dim=0,
                dim_size=graph.num_node * graph.num_relation
            )
            update += edge_update
            
        return update.view(graph.num_node, self.num_relation * self.input_dim).to(device)

    def combine(self, input, update):
        device = input.device
        self.linear.to(device)  # Ensure the linear layers are on the correct device
        self.self_loop.to(device)
        if self.batch_norm:
            self.batch_norm.to(device)
        
        output = self.linear(update) + self.self_loop(input)
        if self.batch_norm:
            output = self.batch_norm(output)
        if self.activation:
            output = self.activation(output)
        return output

    def forward(self, graph, input, new_edge_list=None, new_edge_weight=None):
        """
        Perform message passing over the graph(s).

        Parameters:
            graph (Graph): graph(s)
            input (Tensor): node representations of shape :math:`(|V|, ...)`
        """
        if self.gradient_checkpoint:
            update = checkpoint.checkpoint(self.message_and_aggregate, graph, input)
        else:
            update = self.message_and_aggregate(graph, input, new_edge_list)
        output = self.combine(input, update)
        return output

#### 测试

In [16]:
input_dim = graph.node_feature.shape[-1]
output_dim = 512
num_relations = graph.num_relation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


new_node_feature = RewireGearnet(input_dim, output_dim, num_relations)(graph, graph.node_feature.to(device).float(), attn_output).to(device)

print(new_node_feature.shape)
print(new_node_feature)

torch.Size([600, 512])
tensor([[0.0000, 0.4906, 0.2537,  ..., 0.0000, 0.0000, 0.0000],
        [0.4202, 0.9803, 0.0000,  ..., 0.4804, 0.3370, 0.0000],
        [0.2416, 0.7412, 0.0000,  ..., 0.8642, 0.6196, 0.0000],
        ...,
        [0.7878, 1.0133, 0.0000,  ..., 0.8846, 0.0000, 0.0000],
        [0.0000, 0.5266, 0.4165,  ..., 0.5259, 0.3598, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<ReluBackward0>)


# 最终模型

In [17]:
class DGMGearnet(nn.Module, core.Configurable):

    def __init__(self, input_dim, hidden_dims, num_relation, num_heads, window_size, k, edge_input_dim=None, num_angle_bin=None,
                 short_cut=False, batch_norm=False, activation="relu", concat_hidden=False, readout="sum"):
        super(DGMGearnet, self).__init__()

        #if not isinstance(hidden_dims, Sequence):
            #hidden_dims = [hidden_dims]
        self.input_dim = input_dim
        self.output_dim = sum(hidden_dims) if concat_hidden else hidden_dims[-1]
        self.dims = [input_dim] + list(hidden_dims)
        self.num_heads = num_heads
        self.window_size = window_size
        self.k = k
        self.edge_dims = [edge_input_dim] + self.dims[:-1]
        self.num_relation = num_relation
        self.num_angle_bin = num_angle_bin
        self.short_cut = short_cut
        self.concat_hidden = concat_hidden
        self.batch_norm = batch_norm

        self.layers = nn.ModuleList()
        self.score_layers = nn.ModuleList()
        for i in range(len(self.dims) - 1):
            
            self.score_layers.append(relationalGraph(self.dims[i], self.dims[i + 1], num_relation, edge_input_dim, 
                                            batch_norm, activation))
            
            self.score_layers.append(Rewirescorelayer(self.dims[i + 1], self.dims[i + 1], self.num_heads, self.window_size, 
                                            self.k, temperature=0.5, dropout=0.1))
            
            self.layers.append(RewireGearnet(self.dims[i], self.dims[i + 1], num_relation,
                                            edge_input_dim=None, batch_norm=False, activation="relu"))
        
        if num_angle_bin:
            self.spatial_line_graph = layers.SpatialLineGraph(num_angle_bin)
            self.edge_layers = nn.ModuleList()
            for i in range(len(self.edge_dims) - 1):
                self.edge_layers.append(layers.GeometricRelationalGraphConv(
                    self.edge_dims[i], self.edge_dims[i + 1], num_angle_bin, None, batch_norm, activation))

        if batch_norm:
            self.batch_norms = nn.ModuleList()
            for i in range(len(self.dims) - 1):
                self.batch_norms.append(nn.BatchNorm1d(self.dims[i + 1]))

        if readout == "sum":
            self.readout = layers.SumReadout()
        elif readout == "mean":
            self.readout = layers.MeanReadout()
        else:
            raise ValueError("Unknown readout `%s`" % readout)

    def forward(self, graph, input, edge_list=None, all_loss=None, metric=None):
        """
        Compute the node representations and the graph representation(s).

        Parameters:
            graph (Graph): :math:`n` graph(s)
            input (Tensor): input node representations
            all_loss (Tensor, optional): if specified, add loss to this tensor
            metric (dict, optional): if specified, output metrics to this dict

        Returns:
            dict with ``node_feature`` and ``graph_feature`` fields:
                node representations of shape :math:`(|V|, d)`, graph representations of shape :math:`(n, d)`
        """
        hiddens = []
        layer_input = input
        if self.num_angle_bin:
            line_graph = self.spatial_line_graph(graph)
            edge_input = line_graph.node_feature.float()

        for i in range(len(self.layers)):
            
            
            relational_output = self.score_layers[2*i](graph, layer_input, edge_list)
            new_edge_list = self.score_layers[2*i+1](graph, relational_output)
            
            hidden = self.layers[i](graph, layer_input, new_edge_list)
            
            if self.short_cut and hidden.shape == layer_input.shape:
                hidden = hidden + layer_input
                
            if self.num_angle_bin:
                edge_hidden = self.edge_layers[i](line_graph, edge_input)
                edge_weight = graph.edge_weight.unsqueeze(-1)
                if new_edge_list is None:
                    node_out = graph.edge_list[:, 1] * self.num_relation + graph.edge_list[:, 2]
                else:
                    node_out = new_edge_list[:, 1] * self.num_relation + new_edge_list[:, 2]
                
                    update = scatter_add(edge_hidden * edge_weight, node_out, dim=0,
                                        dim_size=graph.num_node * self.num_relation)
                update = update.view(graph.num_node, self.num_relation * edge_hidden.shape[1])
                update = self.layers[i].linear(update)
                update = self.layers[i].activation(update)
                hidden = hidden + update
                edge_input = edge_hidden
            if self.batch_norm:
                hidden = self.batch_norms[i](hidden)
                
            hiddens.append(hidden)
            layer_input = hidden
            edge_list = new_edge_list

        if self.concat_hidden:
            node_feature = torch.cat(hiddens, dim=-1)
        else:
            node_feature = hiddens[-1]
        graph_feature = self.readout(graph, node_feature)
        print("node_feature: ", node_feature.shape)

        return {
            "graph_feature": graph_feature,
            "node_feature": node_feature
        }

### 测试

In [18]:
input_dim = graph.node_feature.shape[-1]
hidden_dims = [512, 512, 512, 512, 512]
num_relations = graph.num_relation
num_heads = 8
window_size = 50
k = 5


output = DGMGearnet(input_dim, hidden_dims, num_relations, num_heads, window_size, k).to(device)(graph.to(device), graph.node_feature.to(device).float())

node_feature:  torch.Size([600, 512])


In [19]:
print(output["node_feature"])
print(output["node_feature"].shape)
print("\n")

print(output["graph_feature"])
print(output["graph_feature"].shape)
torch.cuda.empty_cache()

tensor([[ 10.6871,   0.0000,   0.0000,  ...,   0.0000,  30.8024,   0.0000],
        [  0.0000,   0.0000,   9.9722,  ...,   6.0842,  27.4922,  23.0079],
        [  0.0000,   0.0000,  37.9458,  ...,  40.1465, 118.7449,  22.7782],
        ...,
        [ 71.0768,   0.0000,   0.0000,  ...,  12.7928,   0.0000,   0.0000],
        [  0.0000,   0.0000,   8.7190,  ...,   5.3802,  54.0361,   9.6626],
        [  0.0000,   0.0000,   2.7798,  ...,   1.5191,  11.1723,   0.0000]],
       device='cuda:0', grad_fn=<ReluBackward0>)
torch.Size([600, 512])


tensor([[1667.0217,  445.2009,  418.8975,  ..., 1931.7217, 4053.8359,
         1067.4402],
        [4312.1523,  880.1165, 1152.3745,  ..., 4668.5718, 9066.7549,
         2825.6218]], device='cuda:0', grad_fn=<ScatterAddBackward>)
torch.Size([2, 512])


# 测试

In [20]:
import torch
from torch import nn
import torch.nn.functional as F

class Rewirescorelayer2(nn.Module):
    def __init__(self, in_features, out_features, num_heads, window_size, k, temperature=0.5, dropout=0.1):
        super(Rewirescorelayer2, self).__init__()
        self.num_heads = num_heads
        self.out_features = out_features
        self.window_size = window_size
        self.temperature = temperature
        self.dropout = nn.Dropout(dropout)
        self.k = k

        self.query = nn.Linear(in_features, out_features * num_heads)
        self.key = nn.Linear(in_features, out_features * num_heads)
        self.value = nn.Linear(in_features, out_features * num_heads)
        self.scale = torch.sqrt(torch.FloatTensor([out_features])).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

    def get_start_end(self, current, graph):
        segment = graph.num_nodes.repeat(graph.num_relation)
        index = torch.zeros(segment.size(0) + 1, dtype=segment.dtype)

        for i in range(1, len(index)):
            index[i] = index[i - 1] + segment[i - 1]

        for i in range(len(index) - 1):
            if index[i] <= current < index[i + 1]:
                return (index[i].item(), index[i + 1].item())
            elif index[i] == current:
                return (index[i].item(), index[i + 1].item())

        if current == index[-1]:
            return (index[-1].item(), index[-1].item())

    def gumbel_softmax_top_k(self, logits, tau=1.0, hard=False):
        gumbels = -torch.empty_like(logits).exponential_().log()
        gumbels = (logits + gumbels) / tau

        y_soft = F.softmax(gumbels, dim=-1)

        if hard:
            topk_indices = logits.topk(self.k, dim=-1)[1]
            y_hard = torch.zeros_like(logits).scatter_(-1, topk_indices, 1.0)
            y = (y_hard - y_soft).detach() + y_soft
        else:
            y = y_soft

        return y

    def forward(self, graph, node_features):
        device = node_features.device
        num_nodes = node_features.size(0)
        half_window = self.window_size // 2

        Q = self.query(node_features).view(num_nodes, self.num_heads, self.out_features)
        K = self.key(node_features).view(num_nodes, self.num_heads, self.out_features)

        output = torch.zeros(num_nodes, num_nodes, device=device)

        start_end_indices = [self.get_start_end(i, graph) for i in range(num_nodes)]

        # Precompute K_windows
        K_windows_padded = torch.zeros((num_nodes, self.window_size, self.num_heads, self.out_features), device=device)
        for i in range(num_nodes):
            start_index, end_index = start_end_indices[i]
            start = max(start_index, i - half_window)
            end = min(end_index, i + half_window)
            K_window = K[start:end]
            K_windows_padded[i, :K_window.size(0)] = K_window
        

        scores = torch.einsum("nhd,nmhd->nhm", Q, K_windows_padded) / self.scale
        scores = scores / self.temperature

        attention_weights = F.softmax(scores, dim=-1).mean(dim=1)  # [num_nodes, max_window_size, out_features]
        print(attention_weights.shape)

        for i in range(num_nodes):
            start_index, end_index = start_end_indices[i]
            start = max(start_index, i - half_window)
            end = min(end_index, i + half_window )
            output[i, start:end] = attention_weights[i, :end-start]

        edge_list = self.gumbel_softmax_top_k(output, self.temperature, self.k)

        return edge_list



In [29]:
import numpy as np

input_dim = relational_output.shape[-1]
output_dim = 1024
num_heads = 8
window_size = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
k = 5


relational_output = relational_output.to(device)
module = Rewirescorelayer2(input_dim, output_dim, num_heads, window_size, k).to(device)
attn_output = module(graph, relational_output)
print(attn_output.shape)
print(attn_output)

RuntimeError: CUDA out of memory. Tried to allocate 6.41 GiB (GPU 0; 47.54 GiB total capacity; 38.90 GiB already allocated; 926.31 MiB free; 45.11 GiB reserved in total by PyTorch)

In [28]:
torch.cuda.empty_cache()